# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [1]:
import networkx
import urllib2
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

In [2]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A generator of the nodes in the graph, yielding a list of the form:
            ['CH', 'AG, 'ME', ...]
    """
    # TODO: implement function
    i = 0 
    a = []
    for line in gfile:
        if line[0] == '\n':  # break when blank line
            break
        elif line[2] == ' ': #choose lines starting with codenames
            a.append(line[0:2])
    return a         
    pass

Next implement a function to read in the edges from the input file.

In [3]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        A generator of the edges in the graph, yielding a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    l=[] #initialize a list to store edges
    for line in gfile:
        if line[1:2] == ':' or line[2:3] == ':':  #c choose lines which list the groups of characters
            b = line.strip("\n")
            c = b.find(':') #find all the groups of characters that interacted 
            b = b[c+1:]
            a = b.split(';') #groups of characters that interacted
            i = 0
            while i < len(a):
                d = a[i]
                e = d.split(',')
                if len(e) != 1: #skip the groups of single character
                    f = 0
                    while f < len(e): # find all the edges
                        g=f+1
                        while g < len(e):
                            h = (e[f],e[g])
                            l.append(h) #store the edges in the list
                            g = g+1
                        f=f+1
                i=i+1
    return l    
    pass

The following code should now correctly create the graph.

In [4]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [188]:
def Search(graph, root):
    """Runs depth-first search through a graph, starting at a given root. Neighboring
    nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
    rootlist=[]  #initialize a rootlist
    def insearch(graph,root): #then do the search and store the outcomes in the rootlist
        i = 0
        g = graph.nodes() #read all the nodes of the graph
        g.sort() #sort the nodes in the alphabetical order
        rootlist.append(root) #store the root into the list
        while i < len(g): 
            if ((root,g[i]) in G.edges() or (g[i],root) in G.edges()): #find the first neighbor of the root 
                if not (g[i] in rootlist):
                    insearch(graph,g[i]) #do the recursion
            i = i+1
        return rootlist
    insearch(graph,root) #call the insearch function
    return rootlist
    pass   

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [190]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [196]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    connected_components=[] #intialize a list to store the connected components
    visited=[] #initialize a list to store the nodes of the connected components
    i = 0 
    g = graph.nodes() #read all the nodes of the graph
    g.sort() #sort the nodes
    while i < len(g):
        if not (g[i] in visited): # find a node that has not been visited according to the alphabetical order
            Search(graph,g[i]) #search this node
            a=Search(graph,g[i])
            t=0
            while t < len(a): #store the connected nodes of this node
               visited.append(a[t])
               t = t+1
            connected_components.extend([a]) #store the connected components of this node as a list
        i = i+1
    return connected_components
    pass

We will check correctness of your code by verifying that your output list is identical to our solution.

In [197]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [198]:
component_sizes = [len(c) for c in character_interactions]
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True
